In [1]:
from model import FinancialLSTMModel
import pandas as pd

2025-11-15 21:04:16.020549: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-15 21:04:16.090234: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-15 21:04:17.976329: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
FEATURES = [
    ("Close", "standard"),
    ("Volume", "minmax"),
    ("rsi_14", "minmax"),
    ("rsi_28", "minmax"),
    ("rsi_7", "minmax"),
    ("macd", "standard"),
    ("ema_20", "standard"),
    ("ema_50", "standard"),
    ("stoch_k", "minmax"),
    ("stoch_d", "minmax"),
    ("roc", "standard"),
    ("adx", "minmax"),
    ("di_plus", "minmax"),
    ("di_minus", "minmax"),
    ("atr", "standard"),
    ("close_pos", "none"),
    ("body_range_ratio", "none"),
]

REPS = 100
OUTPUT='./../res/val_05.csv'

In [ ]:
res = []
for rep in range(REPS):
    model = FinancialLSTMModel(
    csv_path='./../data/GOOGL.csv',
    date_col="Date",
    features=FEATURES,
    target="direction",
    seq_length=60,
    batch_size=32,
    learning_rate=0.001,
    epochs=100,
    val_split=0.05,
    shuffle=False,
    training_ranges=[('2023-01-01', '2025-09-30')],
    testing_ranges=[('2025-10-01', '2025-12-01')]
    )

    print(f"--- REPETITION {rep + 1} ---")
    
    model.prepare_data()
    model.build_model()
    model.train()
    results = model.evaluate()
    print(results)
    print()
    
    res.append(results)
    df_res = pd.DataFrame(res)
    df_res.to_csv(OUTPUT, index=False)    
    

--- REPETITION 1 ---
Training samples: 654, Validation samples: 35, Testing samples: 33


2025-11-15 21:04:20.218303: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


{'first_prediction_correct': np.False_, 'accuracy': 0.3939393939393939, 'f1_score': 0.0, 'confusion_matrix': [[13, 0], [20, 0]]}

--- REPETITION 2 ---
Training samples: 654, Validation samples: 35, Testing samples: 33
{'first_prediction_correct': np.False_, 'accuracy': 0.3939393939393939, 'f1_score': 0.0, 'confusion_matrix': [[13, 0], [20, 0]]}

--- REPETITION 3 ---
Training samples: 654, Validation samples: 35, Testing samples: 33
{'first_prediction_correct': np.False_, 'accuracy': 0.3939393939393939, 'f1_score': 0.0, 'confusion_matrix': [[13, 0], [20, 0]]}

--- REPETITION 4 ---
Training samples: 654, Validation samples: 35, Testing samples: 33
{'first_prediction_correct': np.False_, 'accuracy': 0.3939393939393939, 'f1_score': 0.0, 'confusion_matrix': [[13, 0], [20, 0]]}

--- REPETITION 5 ---
Training samples: 654, Validation samples: 35, Testing samples: 33
{'first_prediction_correct': np.False_, 'accuracy': 0.3939393939393939, 'f1_score': 0.0, 'confusion_matrix': [[13, 0], [20, 0]]}